## Homework 1 Solutions


#### Part 1: SAS Syntax
1. How many total statements is in the following code? How many DATA steps? How many PROC steps?

In [ ]:
data sales_data;
    set company_sales;
    where region = 'North';
    total_sales = units_sold * price_per_unit;
run;

proc means data=sales_data;
    var total_sales;
run;

proc print data=sales_data;
    where total_sales > 10000;
run;

:::{admonition} Solution
:class: Tip

There are 11 statements total, 1 DATA step and 2 PROC steps.

:::

2 a).Which of the following is a valid variable name?
- total_sales
- 2nd_quarter
- _profit
- annual-sales

b). Modify any invalid names to be valid according to the SAS variable naming guidelines.

:::{admonition} Solution
:class: Tip

`total_sales` and `_profit` are the only valid variable name. `2nd_quarter` is invalid due to it starting with a numeric value and `annual-sales` is invalid due to the `-`. 
- `2nd quarter` -> `sec_quarter`
- `annual-sales` -> `annual_sales`

:::

3). Identify the syntax errors in the code below. You may review the log as well. Fix the code then rerun the program. What do the outputs look like? Create titles for your outputs by adding in a statement before the `proc` statements such as: `title "insert_table_title_here"`

```
daat mycars;
    set sashelp.cars;
	AvgMPG=mean(mpg_city, mpg_highway);
run;

proc print data=mycars
	var make model type avgmpg;
	where AvgMPG > 35

proc means data=mycars average min max maxdec=1;
	var avgmpg;
	class type;
run;
```

:::{admonition} Solution
:class: Tip

- Line 1: `data` is mispelled as `daat`
- A `;` is missing after the `proc print` statement and after the `where` statement of the same step.
- The second `proc print` statement is missing the ending `run;`.
- The `proc means` statement should have `mean` instead of average.
  
(see corrected code below)

```
data mycars;
    set sashelp.cars;
	AvgMPG=mean(mpg_city, mpg_highway);
run;

proc print data=mycars;
	var make model type avgmpg;
	where AvgMPG > 35;
run;

proc means data=mycars mean min max maxdec=1;
	var avgmpg;
	class type;
run;

```

Add titles to the report:

```
data mycars;
    set sashelp.cars;
    AvgMPG = mean(mpg_city, mpg_highway);
run;

title 'Cars with an average MPG greater than 35`;
proc print data=mycars;
	var make model type avgmpg;
	where AvgMPG > 35;
run;

title `Distribution of average MPG variable`;
proc means data=mycars mean min max maxdec=1;
    var avgmpg;
    class type;
run;

/* re-initialize title at the end */
title;

```

:::

### Part 2: Working with Data
4). Import the `storm.xlsx` into a library called `hw1`. After impoting the excel sheet into SAS, you can reference the data by calling its respective *sheet name*. (ie: `hw1.Storm_Summary`)

5). Summarize the entire dataset.

6). Check the data for missing values or duplicates. If there are duplicates, continue the assignment without the duplicate data.

7). Create SAS code to answer these questions: 
- How many storms were there in 2005? in 2016?
- For the storms in 2016, what was the average MaxWindMPH? Are there any extreme values?
- Concatenate the columns `HEM NS` and `HEM EW` to create a new column `Hem`. For all years, which hempishere `HEM` had the most storms? 
- Create a new column that categorizes `MinPressure` into these categories: 900 and below: Low, 901-1000: Medium, over 1000: High
- Provide summary statistics for `MinPressure` and frequency tables for your new categorized column.
- Format all Date columns so that it shows as: Weekday, Month Day, Year.
- Format the `Name` column so that the storm names are capitalized for the first letter and lowercase for everything else.
- Sort the data by StartDate.

8). Save the modified `storm` table as a SAS table.

9). Some of the storms that had a significant impact on Guam include Karen, Pamela, Omar, Paka, Pongsona, and Mawar. Filter the dataset to include only these storms (note that some may not be present). Save the filtered data as a SAS table called `guam_storms.sas7bdat`. Next, calculate the average `MaxWindMPH` and `MinPressure` for these storms and give the output the title "Average Values for Severe Guam Storms." Use the `ods output` function to save this result as a SAS table. (see example of `ods output` below)

```
ods output ExtremeObs=extreme_vals;
proc univariate data=ma505.overwatch;
    var Damage;
run;

/* save to MA505 library */
data ma505.extreme_vals;
    set extreme_vals;
run;
```


### Part 2 Solutions

:::{admonition} IMPORTANT
:class: Important

My local version of `SAS (SAS 9.4)` has issues with the `XLSX` engine, so I will be importing the data for Part 2 as a csv instead. HOWEVER, to import the dataset you should use the statement:

`LIBNAME hw1 XLSX "/home/u63936157/hw1/storm.xlsx";` and refer to the SAS table as `hw1.storm_summary`.

:::

#### 4). Import the `storm.xlsx` file

In [34]:
/* Import the `storm.xlsx` into a library called `hw1`. 
After impoting the excel sheet into SAS, you can reference the data 
by calling its respective *sheet name*. (ie: `hw1.Storm_Summary`) */

libname hw1 "datasets/hw1/";

proc import datafile="datasets/hw1/storm.csv" dbms=csv 
		out=hw1.storm_summary;
run;

71                                                         The SAS System                            16:03 Monday, November 18, 2024

1813       ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
1813     ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
1814       
1815       libname hw1 "datasets/hw1/";
NOTE: Libref HW1 was successfully assigned as follows: 
      Engine:        V9 
      Physical Name: C:\Users\rdominguez\datasets\hw1
1816       
1817       proc import datafile="datasets/hw1/storm.csv" dbms=csv
1818       		out=hw1.storm_summary;
1819       run;

1820        /**********************************************************************
1821        *   PRODUCT:   SAS
1822        *   VERSION:   9.4
1823        *   CREATOR:   External File Interface
1824        *   DATE:      18NOV24
1825        *   DESC:      Generated SAS Datastep Code
1826        *   TEMPLATE SOURC

#### 5). Summarize the entire dataset.

In [44]:
/* 5). Summarize the entire dataset. */

/* start with summarizing the dataset itself */
proc contents data=hw1.storm_summary;
run;


The above output shows that there are 3118 rows with 12 different variables.

In [51]:
/* here i will use the proc means function to just find the mean, min, max, and spread of each variable in the dataset */
title 'Summary of values in dataset';
proc means data = hw1.storm_summary;
run;

/* i will also run proc univariate for ALL numeric variables */
title 'Distribution of numeric values';
proc univariate data = hw1.storm_summary;
run;

#### 6). Check the data for missing values or duplicates. If there are duplicates, continue the assignment without the duplicate data.

In [92]:
/* to check missing values for all numeric variables, we can modify the proc means statement */

proc means data = hw1.storm_summary nmiss;
run;

/* we can use the _character_ to subset to only CHARACTER variables */
proc freq data=hw1.storm_summary;
    tables _character_ / missing;
run;

Our output above shows that there is a total of `315` missing values. `23` from `MaxWindMPH` and `196` from `MinPressure`, `80` from `Name`, and `16` from `Basin`.

In [57]:
/* check for duplicates by ALL variables */
proc sort data=hw1.storm_summary
    out=hw1.storm_summary_clean
    nodupkey
    dupout=hw1.storm_summary_duplicate;
by _all_;
run;

117                                                        The SAS System                            16:03 Monday, November 18, 2024

3020       ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
3020     ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
3021       
3022       /* check for duplicates by ALL variables */
3023       proc sort data=hw1.storm_summary
3024           out=hw1.storm_summary_clean
3025           nodupkey
3026           dupout=hw1.storm_summary_duplicate;
3027       by _all_;
3028       run;

NOTE: There were 3118 observations read from the data set HW1.STORM_SUMMARY.
NOTE: 0 observations with duplicate key values were deleted.
NOTE: The data set HW1.STORM_SUMMARY_CLEAN has 3118 observations and 12 variables.
NOTE: The data set HW1.STORM_SUMMARY_DUPLICATE has 0 observations and 12 variables.
NOTE: PROCEDURE SORT used (Total process time):
      real time

From the log output above, we see that our *duplicate* dataset `hw1.storm_summary_duplicate` has 0 observations, meaning there are no duplicates found! We will proceed with just using the `hw1.storm_summary` dataset.

#### 7). Create SAS code to answer these questions: 
- How many storms were there in 2005? in 2016?

In [100]:
proc freq data=hw1.storm_summary;
    tables Season;
    where Season in (2005, 2016);
run;

Season,Frequency,Percent,CumulativeFrequency,CumulativePercent
2005,102,53.13,102,53.13
2016,90,46.88,192,100.00


In 2005 there was a total of 102 storms and in 2016 90.

- For the storms in 2016, what was the average MaxWindMPH? Are there any extreme values?

In [95]:
proc univariate data=hw1.storm_summary;
    var MaxWindMPH;
    where Season = 2016;
run;

The average `MaxWindMPH` in 2016 is ~79.3, its highest extreme value is 173 and lowest extreme value is 29.

- Concatenate the columns `HEM NS` and `HEM EW` to create a new column `Hem`. For all years, which hempishere `HEM` had the most storms? 


In [106]:
data hw1.storm_summary;
    set hw1.storm_summary;
    HEM = cats("Hem NS"n, "Hem EW"n);
run;

proc freq data = hw1.storm_summary;
    tables HEM;
run;

HEM,Frequency,Percent,CumulativeFrequency,CumulativePercent
NE,1028,32.97,1028,32.97
NW,1143,36.66,2171,69.63
SE,756,24.25,2927,93.87
SW,191,6.13,3118,100.00


The northwest HEM had the most storms.

- Create a new column that categorizes `MinPressure` into these categories: 900 and below: Low, 901-1000: Medium, over 1000: High
-  Provide summary statistics for `MinPressure` and frequency tables for your new categorized column.


In [118]:
data hw1.storm_summary;
    set hw1.storm_summary;
    length PressureCategory $ 10;
    if MinPressure <= 900 then PressureCategory = "Low";
    else if MinPressure > 900 and MinPressure <= 1000 then PressureCategory = "Medium";
    else PressureCategory = "High";
run;

proc means data = hw1.storm_summary;
    var MinPressure;
run;

proc freq data = hw1.storm_summary;
    tables PressureCategory;
run;

With the `proc means` statement for the `MinPressure` variable, we can see the average is ~961 which corresponds with our *Medium* `PressureCategory` having the most observations.

- Format all Date columns so that it shows as: Weekday, Month Day, Year.
- Format the `Name` column so that the storm names are capitalized for the first letter and lowercase for everything else.
- Sort the data by StartDate.

In [129]:
data hw1.storm_summary; 
    set hw1.storm_summary;
     format StartDate WEEKDATE. EndDate WEEKDATE.;
    Name = PROPCASE(Name);
run;

PROC SORT data=hw1.storm_summary out=hw1.storm_summary_sort;
    by StartDate;
run;

proc print data=hw1.storm_summary (obs=10);
run;

Obs,Season,Name,Basin,Type,MaxWindMPH,MinPressure,StartDate,EndDate,Hem NS,Hem EW,Lat,Lon,HEM,PressureCategory
1,1980,,NA,TS,35,.,"Thursday, July 17, 1980","Tuesday, November 18, 1980",N,W,25.7,-91.2,NW,Low
2,1980,Allen,NA,TS,190,899,"Thursday, July 31, 1980","Monday, August 11, 1980",N,W,21.8,-86.4,NW,Low
3,1980,Bonnie,NA,ET,98,975,"Thursday, August 14, 1980","Tuesday, August 19, 1980",N,W,23.7,-38.9,NW,Medium
4,1980,Charley,NA,TS,81,989,"Wednesday, August 20, 1980","Monday, August 25, 1980",N,W,38.9,-66.7,NW,Medium
5,1980,Danielle,NA,TS,58,1004,"Thursday, September 4, 1980","Sunday, September 7, 1980",N,W,29.4,-93.4,NW,High
6,1980,Earl,NA,ET,75,985,"Thursday, September 4, 1980","Thursday, September 11, 1980",N,W,30.4,-42.5,NW,Medium
7,1980,Frances,NA,ET,115,958,"Saturday, September 6, 1980","Sunday, September 21, 1980",N,W,12.8,-29.8,NW,Medium
8,1980,Georges,NA,TS,81,993,"Monday, September 1, 1980","Monday, September 8, 1980",N,W,40.2,-59,NW,Medium
9,1980,Hermine,NA,TS,69,993,"Saturday, September 20, 1980","Friday, September 26, 1980",N,W,17.8,-87.8,NW,Medium
10,1980,Ivan,NA,ET,104,970,"Wednesday, October 1, 1980","Sunday, October 12, 1980",N,W,28.1,-31.1,NW,Medium


#### 9). Some of the storms that had a significant impact on Guam include Karen, Pamela, Omar, Paka, Pongsona, and Mawar. Filter the dataset to include only these storms (note that some may not be present). Save the filtered data as a SAS table called `guam_storms.sas7bdat`. Next, calculate the average `MaxWindMPH` and `MinPressure` for these storms and give the output the title "Average Values for Severe Guam Storms." Use the `ods output` function to save this result as a SAS table. (see example of `ods output` below)

In [158]:
/* manually add in the typo in Paka or - to find it */
%let gu_storms = 'Karen', 'Pamela', 'Omar', 'Paka -', 'Pongsona', 'Mawar';
%let year = 1962, 1976, 1992, 1997, 2002, 2023;

data hw1.guam_storms;
    set hw1.storm_summary;
    where Name in (&gu_storms) and Season in (&year);
run;

proc print data=hw1.guam_storms;
run;

Obs,Season,Name,Basin,Type,MaxWindMPH,MinPressure,StartDate,EndDate,Hem NS,Hem EW,Lat,Lon,HEM,PressureCategory
1,1992,Omar,WP,TS,115,920,"Sunday, August 23, 1992","Wednesday, September 9, 1992",N,E,14.7,140.6,NE,Medium
2,1997,Paka -,WP,TS,115,920,"Friday, November 28, 1997","Tuesday, December 23, 1997",N,E,14.1,140.7,NE,Medium
3,2002,Pongsona,WP,ET,104,940,"Monday, December 2, 2002","Wednesday, December 11, 2002",N,E,14.4,144.6,NE,Medium


In [162]:
ods output Summary="AvgGuam";
title 'Average Values for Severe Guam Storms';
proc means data=hw1.guam_storms;
    var MaxWindMPH MinPressure;
run;

ods output close

Variable,N,Mean,Std Dev,Minimum,Maximum
MaxWindMPH MinPressure,3 3,111.3333333 926.6666667,6.3508530 11.5470054,104.0000000 920.0000000,115.0000000 940.0000000
